In [1]:
#Ignore scikit learn deprication warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re


C:\Users\aparn\Desktop\cs638project\learning


In [37]:
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('train_reduced.csv',delimiter=',')
test = genfromtxt('unlabeled_reduced.csv',delimiter=',')
#unlabeled = genfromtxt('unlabeled_reduced.csv',delimiter=',')
# get rid of first row (I think this is just column labels)
train = train[1:,:]
test = test[1:,:]




#print(test)
#print(test.shape)
#unlabeled = unlabeled[1:,:]

# # # replace missing featureVals with 0.5 (halfway in between)
# where_are_NaNs = np.isnan(train)
# train[where_are_NaNs] = 0


##Drop all nans:
train = train[~np.isnan(train).any(axis=1)]
test = test[~np.isnan(test).any(axis=1)]
#unlabeled = unlabeled[~np.isnan(unlabeled).any(axis=1)]



y = train[:,train.shape[1]-1] # label
X = train[:,1:train.shape[1]-1]  # data




124

In [38]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import *
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *



(124, 17)
(124,)


In [39]:
# CME's grid search stuff
from sklearn.grid_search import GridSearchCV


# SVC tuning
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10,
                       scoring='f1')
clf.fit(X,y)

# Get parameters from model
#params = clf.get_params()
#sv = clf.support_vectors
#nv = clf.n_support_
#a  = clf.dual_coef_
#b  = clf._intercept_
#cs = clf.classes_



prediction = clf.predict(X_test)
print(prediction)

print("SVC:" + str(clf.best_score_))
print("SVC:" + str(clf.best_estimator_))


[ 0.  0.  1. ...,  0.  0.  0.]
SVC:0.8240783410138248
SVC:SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [33]:
_id = pd.DataFrame(ids)
label = pd.DataFrame(prediction)
df = pd.concat([_id, label], axis=1)
df.columns = ['id', 'label']
df = df[df.label != 0]


unlabeled = em.read_csv_metadata("unlabeled.csv", encoding="ISO-8859-1")
unlabeled.rename(columns={'_id':'id'}, inplace=True)
#unlabeled = unlabeled.set_index(['id'])
unlabeled.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,countries_countries_lev_sim,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim
0,2103728,6253,265,light red meranti,Shorea,Dipterocarpaceae,Terrestrial,Indonesia (Sumatera); Malaysia (Peninsular Malaysia);,NaN,['\n The species occurs in some forest reserves.\n\n \n '],...,0.129630,-33.0,2.0,0,0.500000,1.0,0.0,0,0.0,1
1,6657422,907,841,NaN,Aphanius,Cyprinodontidae,Freshwater,Ethiopia;,\r\r\r\r\n It is expected that there will be high human interference in the habitats of this...,"['\n No information available. This species would benefit from protective legislation, as wel...",...,0.250000,-2.0,2.0,1,1.000000,0.0,1.0,0,0.0,1
2,7553756,2846,954,"blackfaced black spider monkey, chamek spider monkey, peruvian black spider monkey, peruvian spi...",Ateles,Atelidae,Terrestrial,"Bolivia, Plurinational States of; Brazil (Acre, Amazonas, Mato Grosso, RondÃ?Â´nia); Peru;",\r\r\r\r\n The major threat is subsistence and market hunting for food (with guns). An additi...,"['\n This species is confirmed, or may occur, in numerous protected areas.', <br/>, <br/>, 'B...",...,0.000000,-87.0,0.0,0,0.000000,1.0,0.0,0,0.0,0
3,4889866,6311,617,NaN,Shorea,Dipterocarpaceae,Terrestrial,"Brunei Darussalam; Indonesia (Kalimantan); Malaysia (Sabah, Sarawak);",\r\r\r\r\n Directly threatened by logging activities because trees do not reach reproductive ...,NaN,...,0.128571,-48.0,9.0,0,0.333333,1.0,0.0,0,1.0,0
4,8371955,5800,1057,"black sea bass, giant sea bass",Stereolepis,Polyprionidae,Marine,"<div>Pacific Ã¢?? eastern central</div>; Mexico (Baja California, Sonora); United States (Califo...","\r\r\r\r\n The major threat is from overfishing; however, the species is protected in Califor...",['\n The fish has been protected in California since 1982 and protected in Mexico since 1992 ...,...,0.039216,-89.0,2.0,0,0.333333,1.0,0.0,1,0.0,0


In [34]:
#df.columns
corrected_ids = []
for index, row in df.iterrows():
    name = str(row['id']) #look here if errors start happening
    name = name.replace('.0','')
    corrected_ids.append(name)
df['id'] = corrected_ids
#result = pd.concat([unlabeled, df], axis=1, join='inner')
#result.head()
#df = df.set_index(['id'])


In [35]:
df.head()


,id,label
2,8371955,1.0
10,2314553,1.0
11,2099459,1.0
32,1522478,1.0
33,6751519,1.0


In [36]:
#df.head()
#df.set_index('id').join(unlabeled.set_index('id'))
#result.head()
#unlabeled.head()
merge(df, unlabeled, left_on='id', right_on='id', how='outer')



NameError: name 'merge' is not defined

In [13]:

#result = pd.concat([unlabeled, df], axis=1, join='inner')
#result.head()
#result.to_csv('result.csv')

In [29]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                       index=[0, 1, 2, 3])

df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
              'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])

df4.head()

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [30]:
 result = pd.concat([df1, df4], axis=1, join='inner')


In [31]:
result.head()

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
